In [20]:
import pandas as pd
import glob

illegal_words = []
extra_words = ["32B", "27b"]
#extra_words = [""]
files_bef = [
    f for f in glob.glob('test/*output*.csv')
    if not any(bad in f for bad in illegal_words)
    and any(extra in f for extra in extra_words)
]
files_bef = sorted(files_bef)

files = []
for file in files_bef:
    if "Qwen" in file:
        files.append(file)
        files.append(file)
        files.append(file)
    else:
        files.append(file)
        files.append(file)
    


for idx, f in enumerate(files):
    print(f"{idx}: {f}")

dfs   = [pd.read_csv(f) for f in files]
#df_true = pd.read_csv('test/true.csv')
#true_labels = df_true["label"].tolist()
num_predictions = len(files)
grouped = {c: [df[c].tolist() for df in dfs] for c in dfs[0].columns}
num_samples = len(grouped["id"][0])

print(grouped.keys()) 
print(num_samples, num_predictions)




0: test/output_Qwen_Qwen3-32B_15shot_1747581147043.csv
1: test/output_Qwen_Qwen3-32B_15shot_1747581147043.csv
2: test/output_Qwen_Qwen3-32B_15shot_1747581147043.csv
3: test/output_Qwen_Qwen3-32B_20shot_1747587374155.csv
4: test/output_Qwen_Qwen3-32B_20shot_1747587374155.csv
5: test/output_Qwen_Qwen3-32B_20shot_1747587374155.csv
6: test/output_Qwen_Qwen3-32B_25shot_1747584750726.csv
7: test/output_Qwen_Qwen3-32B_25shot_1747584750726.csv
8: test/output_Qwen_Qwen3-32B_25shot_1747584750726.csv
9: test/output_google_gemma-3-27b-it_10shot_1747581147043.csv
10: test/output_google_gemma-3-27b-it_10shot_1747581147043.csv
11: test/output_google_gemma-3-27b-it_10shot_1747588262674.csv
12: test/output_google_gemma-3-27b-it_10shot_1747588262674.csv
13: test/output_google_gemma-3-27b-it_20shot_1747581147043.csv
14: test/output_google_gemma-3-27b-it_20shot_1747581147043.csv
15: test/output_google_gemma-3-27b-it_20shot_1747581147047.csv
16: test/output_google_gemma-3-27b-it_20shot_1747581147047.csv
17

In [75]:
predicted_score_ind = []


pos_perc = 30.40/100
neg_perc = 21.46/100
neutr_perc = 48.14/100
predicted_labels = ["positive"]*num_samples
for i in range(num_samples):
    score = 0
    for j in range(num_predictions):
        score += grouped['positive'][j][i]
        score += 0.5 * grouped['neutral'][j][i]
    predicted_score_ind.append((score,i))
predicted_score_ind = sorted(predicted_score_ind)

offset = 0.015
for i in range(round(num_samples*(neg_perc+neutr_perc+offset))):
    predicted_labels[predicted_score_ind[i][1]] = "neutral"
for i in range(round(num_samples*(neg_perc-offset))):
    predicted_labels[predicted_score_ind[i][1]] = "negative"

for i in range(num_samples):
    pos_prob = 0
    neg_prob = 0
    for j in range(num_predictions):
        pos_prob += float(grouped['positive'][j][i])/num_predictions
        neg_prob += float(grouped['negative'][j][i])/num_predictions
    if pos_prob > 0.2 and neg_prob > 0.2:
        predicted_labels[i] = "neutral"




In [23]:
#predicted_labels = grouped['label'][0]

print(predicted_labels.count("positive")/num_samples)
print(predicted_labels.count("neutral")/num_samples)
print(predicted_labels.count("negative")/num_samples)
pd.DataFrame({
    "id":    range(len(predicted_labels)),
    "label": predicted_labels
}).to_csv("submission_ensemble8.csv", index=False)

# 0.3210609990795749
# 0.3077566730817505
# 0.3711823278386746

# Qwen 32B
# 0.3202242490168187
# 0.3207262990544724
# 0.3590494519287089
    

0.3193038239477868
0.31311187348339053
0.3675843025688227


In [25]:
predicted_labels = []
for i in range(num_samples):
    num_pos = 0
    num_neg = 0
    num_neut = 0
    for j in range(num_predictions):
        if grouped['label'][j][i] == "positive":
            num_pos += 1/num_predictions
        if grouped['label'][j][i] == "negative":
            num_neg += 1/num_predictions
        if grouped['label'][j][i] == "neutral":
            num_neut += 1/num_predictions
    if num_pos > max(num_neg, num_neut):
        predicted_labels.append("positive")
    elif num_neg > max(num_pos, num_neut):
        predicted_labels.append("negative")
    else:
        predicted_labels.append("neutral")
    print(num_pos, num_neut, num_neg)

    # for j in range(num_predictions):
    #     num_pos += grouped["positive"][j][i]/num_predictions
    #     num_neut += grouped["neutral"][j][i]/num_predictions
    #     num_neg += grouped["negative"][j][i]/num_predictions
    
    # if num_pos > max(num_neg, num_neut)+0.05:
    #     predicted_labels.append("positive")
    # elif num_neg > max(num_pos, num_neut)+0.05:
    #     predicted_labels.append("negative")
    # else:
    #     predicted_labels.append("neutral")


num_diff = 0
num_diff_pos_neg = 0
for i in range(num_samples):
    labels = set([grouped["label"][j][i] for j in range(num_predictions)])
    if len(labels) == 3:
        num_diff += 1
        print(grouped["sentence"][0][i], predicted_labels[i], grouped["label"][0][i], grouped["label"][1][i] ,grouped["label"][2][i])
  
        
        
print(num_diff/num_samples, num_diff)

0.15789473684210525 0.8421052631578945 0
0.9999999999999996 0 0
0 0.9999999999999996 0
0 0.42105263157894735 0.5789473684210527
0 0 0.9999999999999996
0.3157894736842105 0.6842105263157894 0
0 0 0.9999999999999996
0 0 0.9999999999999996
0.9999999999999996 0 0
0.9999999999999996 0 0
0 0.9999999999999996 0
0 0.7368421052631577 0.2631578947368421
0 0 0.9999999999999996
0.9999999999999996 0 0
0.9999999999999996 0 0
0 0.7894736842105261 0.21052631578947367
0 0.9999999999999996 0
0 0 0.9999999999999996
0 0 0.9999999999999996
0.9999999999999996 0 0
0 0.47368421052631576 0.5263157894736842
0.9999999999999996 0 0
0 0.9999999999999996 0
0.9999999999999996 0 0
0 0 0.9999999999999996
0 0.9999999999999996 0
0.8421052631578945 0.15789473684210525 0
0.42105263157894735 0.5789473684210527 0
0.6842105263157894 0.3157894736842105 0
0 0.8421052631578945 0.15789473684210525
0 0 0.9999999999999996
0.6842105263157894 0.3157894736842105 0
0 0.15789473684210525 0.8421052631578945
0 0 0.9999999999999996
0.9999